In [ ]:
# ResNet model
from ca_utils import ResNet, BasicBlock
model = ResNet(block=BasicBlock, layers=[1, 1, 1], num_classes=10)  # change num_classes if needed, this is an example

# Dataset
from torchvision import transforms, datasets

import os
if not os.path.exists('EXCV10.zip'):
    !unzip -q EXCV10.zip

# Vanilla image transform
image_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=8, padding_mode='reflect'),               # Random Position Crop
    transforms.RandomHorizontalFlip(),                  # right and left flip
    transforms.ToTensor(),                              # change [0,255] Int value to [0,1] Float value
    transforms.Normalize(mean=(0.4914, 0.4824, 0.4467), # RGB Normalize MEAN
                         std=(0.2471, 0.2436, 0.2616))  # RGB Normalize Standard Deviation
])

# Dataset
import torchvision
train_data = torchvision.datasets.ImageFolder('train/', transform=image_transform)

# Data loader
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## create model and optimizer
learning_rate = 0.0001
weight_decay = 0.0005
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

# make the parameters trainable
for param in model.parameters():
    param.requires_grad = True
# define optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
from tqdm import tqdm_notebook as tqdm

# Train CNN
def train_cnn(model, train_loader):
    loss = AverageMeter()
    model.train()
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)  
        output = model(data) 
        loss_this = F.cross_entropy(output, target)
        optimizer.zero_grad()
        loss_this.backward()
        optimizer.step()
        loss.update(loss_this.item(), target.shape[0])
    print('Train: Average loss: {:.4f}\n'.format(loss.avg))
#torch.save(model.state_dict(),'data/weights_resnet.pth')

In [ ]:
# Dataset
from PIL import Image
from torchvision import transforms, datasets
class EXCV10TestImageFolder(datasets.ImageFolder):
    def __init__(self, *args, **kwargs):
        super(EXCV10TestImageFolder, self).__init__(*args, **kwargs)
        
    def __getitem__(self, index):
        img_path = self.imgs[index][0]
        pic = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            img = self.transform(pic)
        return img

# Vanilla image transform
image_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
# Dataset
test_data = EXCV10TestImageFolder('val/', transform=image_transform)

# Data loader
from torch.utils.data import DataLoader
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
# Test CNN
def test_cnn(model, test_loader):    
    #from ca_utils import ResNet, BasicBlock

    pred_arr = []
    model.eval()

    with torch.no_grad():
        for data in test_loader:          

          img = data
          img = img.to(device)
          output = model(img)

          _, predict_result = torch.max(output.data, dim=1)  
          predict_result = output.argmax(dim=1, keepdim=True)
          predict_result = predict_result.cpu().numpy().flatten()
          for predict in predict_result:
            pred_arr.append(predict)   
    return np.array(pred_arr)

Face Mask Detection

In [ ]:
# Dataset
import os, glob
from PIL import Image
from torch.utils.data import Dataset
class MaskedFaceTestDataset(Dataset):
    def __init__(self, root, transform=None):
        super(MaskedFaceTestDataset, self).__init__()
        self.imgs = sorted(glob.glob(os.path.join(root, '*.png')))
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.imgs[index]
        img = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img

    def __len__(self):
        return len(self.imgs)

In [ ]:
def count_masks(test_dataset):
    
    import torch
    import torchvision    
    import torchvision.transforms as transforms
    from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

    num_classes = 4
    with_mask = 0
    without_mask = 0
    incorrect_mask= 0
    mask_list=[]
    output_list=[]

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 
    
    """ Download the  model 'face_model.pth' from google drive """
    !pip install -U --no-cache-dir gdown --pre
    !gdown --id 1viTuPLNtB5fMhTrk9Pg_YD0N3jyD-TY_

    model.load_state_dict(torch.load('face_model.pth',map_location=torch.device('cpu')))    
    model = model.cpu()
    model.eval()  

    for i in range(len(test_dataset)):
        img= test_dataset[i]

        with torch.no_grad():
            predictions = model([img])

        keep_boxes   = torchvision.ops.nms(predictions[0]['boxes'].cpu(),predictions[0]['scores'].cpu(),0.3)
        score_filter = predictions[0]['scores'].cpu().numpy()[keep_boxes] > 0.8
        test_labels  = predictions[0]['labels'].cpu().numpy()[keep_boxes][score_filter]

        without_mask   += np.count_nonzero(test_labels == 1)
        with_mask      += np.count_nonzero(test_labels == 2)        
        incorrect_mask += np.count_nonzero(test_labels == 3)

        mask_list=[with_mask,without_mask,incorrect_mask]
        output_list.append(mask_list)
    return np.array(output_list, dtype=np.int64)

In [ ]:
model = ResNet(block=BasicBlock, layers=[1, 1, 1], num_classes=10)
cp = torch.load("data/weights_resnet.pth", map_location=torch.device("cpu"))
model.load_state_dict(cp)

output = test_cnn(model, test_loader)
output_2 = count_masks(test_data)